In [1]:
import pandas as pd
import numpy as np

In [2]:
!pip install catboost
!pip install shap
!pip install pyLDAvis
!pip install category_encoders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.6 MB 1.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 569 kB 5.0 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.7 MB 5.2 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136898 sha256=23315c24a673ef96f717d485e2a8b5d0897df0d82c963a764fc0163e1b607a89
  Stored in directory: /root/.cache/pip/wheels/c9/21/f6/17bcf2667e8a68532ba2fbf6d5c72fdf4c7f7d9abfa4852d2f
  Created wheel for sklearn: filename=sklearn-0.0.post1-py3-none-any.whl size=2344 sha256=94b33f

In [3]:
train = pd.read_csv('https://datahack-prod.s3.amazonaws.com/train_file/train_qWM28Yl.csv')
test = pd.read_csv('https://datahack-prod.s3.amazonaws.com/test_file/test_zo1G9sv.csv')

In [4]:
train.head()

,policy_id,policy_tenure,age_of_car,age_of_policyholder,area_cluster,population_density,make,segment,model,fuel_type,...,is_brake_assist,is_power_door_locks,is_central_locking,is_power_steering,is_driver_seat_height_adjustable,is_day_night_rear_view_mirror,is_ecw,is_speed_alert,ncap_rating,is_claim
0,ID00001,0.515874,0.05,0.644231,C1,4990,1,A,M1,CNG,...,No,No,No,Yes,No,No,No,Yes,0,0
1,ID00002,0.672619,0.02,0.375000,C2,27003,1,A,M1,CNG,...,No,No,No,Yes,No,No,No,Yes,0,0
2,ID00003,0.841110,0.02,0.384615,C3,4076,1,A,M1,CNG,...,No,No,No,Yes,No,No,No,Yes,0,0
3,ID00004,0.900277,0.11,0.432692,C4,21622,1,C1,M2,Petrol,...,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,2,0
4,ID00005,0.596403,0.11,0.634615,C5,34738,2,A,M3,Petrol,...,No,Yes,Yes,Yes,No,Yes,Yes,Yes,2,0


In [5]:
train.columns

Index(['policy_id', 'policy_tenure', 'age_of_car', 'age_of_policyholder',
       'area_cluster', 'population_density', 'make', 'segment', 'model',
       'fuel_type', 'max_torque', 'max_power', 'engine_type', 'airbags',
       'is_esc', 'is_adjustable_steering', 'is_tpms', 'is_parking_sensors',
       'is_parking_camera', 'rear_brakes_type', 'displacement', 'cylinder',
       'transmission_type', 'gear_box', 'steering_type', 'turning_radius',
       'length', 'width', 'height', 'gross_weight', 'is_front_fog_lights',
       'is_rear_window_wiper', 'is_rear_window_washer',
       'is_rear_window_defogger', 'is_brake_assist', 'is_power_door_locks',
       'is_central_locking', 'is_power_steering',
       'is_driver_seat_height_adjustable', 'is_day_night_rear_view_mirror',
       'is_ecw', 'is_speed_alert', 'ncap_rating', 'is_claim'],
      dtype='object')

In [6]:
train['policy_id'].duplicated().sum()

0

In [7]:
train.shape

(58592, 44)

In [8]:
train['policy_tenure'].describe()

count    58592.000000
mean         0.611246
std          0.414156
min          0.002735
25%          0.210250
50%          0.573792
75%          1.039104
max          1.396641
Name: policy_tenure, dtype: float64

In [9]:
train.isnull().sum()

policy_id                           0
policy_tenure                       0
age_of_car                          0
age_of_policyholder                 0
area_cluster                        0
population_density                  0
make                                0
segment                             0
model                               0
fuel_type                           0
max_torque                          0
max_power                           0
engine_type                         0
airbags                             0
is_esc                              0
is_adjustable_steering              0
is_tpms                             0
is_parking_sensors                  0
is_parking_camera                   0
rear_brakes_type                    0
displacement                        0
cylinder                            0
transmission_type                   0
gear_box                            0
steering_type                       0
turning_radius                      0
length      

In [10]:
train['age_of_car'].describe()

count    58592.000000
mean         0.069424
std          0.056721
min          0.000000
25%          0.020000
50%          0.060000
75%          0.110000
max          1.000000
Name: age_of_car, dtype: float64

In [11]:
train['age_of_policyholder'].describe()

count    58592.000000
mean         0.469420
std          0.122886
min          0.288462
25%          0.365385
50%          0.451923
75%          0.548077
max          1.000000
Name: age_of_policyholder, dtype: float64

In [12]:
train['area_cluster'].value_counts(normalize=True)*100

C8     23.303523
C2     12.530721
C5     11.911182
C3     10.412684
C14     6.246587
C13     5.842094
C10     5.384694
C9      4.666166
C7      3.698457
C12     2.711974
C1      2.505461
C11     2.068542
C19     1.624795
C6      1.518979
C15     1.315879
C4      1.134967
C17     0.839705
C16     0.684394
C21     0.646846
C18     0.413026
C22     0.353291
C20     0.186032
Name: area_cluster, dtype: float64

In [13]:
train['population_density'].describe()

count    58592.000000
mean     18826.858667
std      17660.174792
min        290.000000
25%       6112.000000
50%       8794.000000
75%      27003.000000
max      73430.000000
Name: population_density, dtype: float64

In [14]:
train['make'].value_counts()

1    38126
3    14018
2     2373
5     2114
4     1961
Name: make, dtype: int64

In [15]:
train['segment'].value_counts()

B2         18314
A          17321
C2         14018
B1          4173
C1          3557
Utility     1209
Name: segment, dtype: int64

In [16]:
train.columns

Index(['policy_id', 'policy_tenure', 'age_of_car', 'age_of_policyholder',
       'area_cluster', 'population_density', 'make', 'segment', 'model',
       'fuel_type', 'max_torque', 'max_power', 'engine_type', 'airbags',
       'is_esc', 'is_adjustable_steering', 'is_tpms', 'is_parking_sensors',
       'is_parking_camera', 'rear_brakes_type', 'displacement', 'cylinder',
       'transmission_type', 'gear_box', 'steering_type', 'turning_radius',
       'length', 'width', 'height', 'gross_weight', 'is_front_fog_lights',
       'is_rear_window_wiper', 'is_rear_window_washer',
       'is_rear_window_defogger', 'is_brake_assist', 'is_power_door_locks',
       'is_central_locking', 'is_power_steering',
       'is_driver_seat_height_adjustable', 'is_day_night_rear_view_mirror',
       'is_ecw', 'is_speed_alert', 'ncap_rating', 'is_claim'],
      dtype='object')

In [17]:
train['model'].value_counts()

M1     14948
M4     14018
M6     13776
M8      4173
M7      2940
M3      2373
M9      2114
M5      1598
M10     1209
M2      1080
M11      363
Name: model, dtype: int64

In [18]:
train['fuel_type'].value_counts()

Petrol    20532
CNG       20330
Diesel    17730
Name: fuel_type, dtype: int64

In [19]:
train['is_claim'].value_counts(normalize=True)

0    0.936032
1    0.063968
Name: is_claim, dtype: float64

In [20]:
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier
from sklearn.metrics import f1_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation as LDA
import pyLDAvis.sklearn
from itertools import combinations
from category_encoders import * 
from sklearn.model_selection import train_test_split

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable
/usr/local/lib/python3.7/dist-packages/past/builtins/misc.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Mapping


In [21]:
train.dtypes

policy_id                            object
policy_tenure                       float64
age_of_car                          float64
age_of_policyholder                 float64
area_cluster                         object
population_density                    int64
make                                  int64
segment                              object
model                                object
fuel_type                            object
max_torque                           object
max_power                            object
engine_type                          object
airbags                               int64
is_esc                               object
is_adjustable_steering               object
is_tpms                              object
is_parking_sensors                   object
is_parking_camera                    object
rear_brakes_type                     object
displacement                          int64
cylinder                              int64
transmission_type               

In [22]:
X = train.drop(columns = ['policy_id', 'is_claim'])
y = train['is_claim']

In [23]:
X

,policy_tenure,age_of_car,age_of_policyholder,area_cluster,population_density,make,segment,model,fuel_type,max_torque,...,is_rear_window_defogger,is_brake_assist,is_power_door_locks,is_central_locking,is_power_steering,is_driver_seat_height_adjustable,is_day_night_rear_view_mirror,is_ecw,is_speed_alert,ncap_rating
0,0.515874,0.05,0.644231,C1,4990,1,A,M1,CNG,60Nm@3500rpm,...,No,No,No,No,Yes,No,No,No,Yes,0
1,0.672619,0.02,0.375000,C2,27003,1,A,M1,CNG,60Nm@3500rpm,...,No,No,No,No,Yes,No,No,No,Yes,0
2,0.841110,0.02,0.384615,C3,4076,1,A,M1,CNG,60Nm@3500rpm,...,No,No,No,No,Yes,No,No,No,Yes,0
3,0.900277,0.11,0.432692,C4,21622,1,C1,M2,Petrol,113Nm@4400rpm,...,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,2
4,0.596403,0.11,0.634615,C5,34738,2,A,M3,Petrol,91Nm@4250rpm,...,No,No,Yes,Yes,Yes,No,Yes,Yes,Yes,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58587,0.355089,0.13,0.644231,C8,8794,2,A,M3,Petrol,91Nm@4250rpm,...,No,No,Yes,Yes,Yes,No,Yes,Yes,Yes,2
58588,1.199642,0.02,0.519231,C14,7788,1,A,M1,CNG,60Nm@3500rpm,...,No,No,No,No,Yes,No,No,No,Yes,0
58589,1.162273,0.05,0.451923,C5,34738,1,A,M1,CNG,60Nm@3500rpm,...,No,No,No,No,Yes,No,No,No,Yes,0
58590,1.236307,0.14,0.557692,C8,8794,1,B2,M6,Petrol,113Nm@4400rpm,...,No,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,2


In [24]:
cat_features = X.select_dtypes(include = 'object').columns.tolist()

In [25]:
for col in cat_features:
  print(train[col].nunique())

22
6
11
3
9
9
11
2
2
2
2
2
2
2
3
2
2
2
2
2
2
2
2
2
2
2
2


In [26]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size =0.2, stratify = y,random_state=42)

In [27]:
y_train.value_counts()

0    43875
1     2998
Name: is_claim, dtype: int64

In [28]:
43875/ 2998

14.634756504336224

In [29]:
from numpy import argmax
from sklearn.datasets import make_classification
from sklearn.metrics import precision_recall_curve
from matplotlib import pyplot

In [30]:
import shap

In [31]:
from sklearn.model_selection import StratifiedKFold

In [32]:
train.columns

Index(['policy_id', 'policy_tenure', 'age_of_car', 'age_of_policyholder',
       'area_cluster', 'population_density', 'make', 'segment', 'model',
       'fuel_type', 'max_torque', 'max_power', 'engine_type', 'airbags',
       'is_esc', 'is_adjustable_steering', 'is_tpms', 'is_parking_sensors',
       'is_parking_camera', 'rear_brakes_type', 'displacement', 'cylinder',
       'transmission_type', 'gear_box', 'steering_type', 'turning_radius',
       'length', 'width', 'height', 'gross_weight', 'is_front_fog_lights',
       'is_rear_window_wiper', 'is_rear_window_washer',
       'is_rear_window_defogger', 'is_brake_assist', 'is_power_door_locks',
       'is_central_locking', 'is_power_steering',
       'is_driver_seat_height_adjustable', 'is_day_night_rear_view_mirror',
       'is_ecw', 'is_speed_alert', 'ncap_rating', 'is_claim'],
      dtype='object')

In [33]:
train.pivot_table(index = 'model', columns = ['fuel_type'], aggfunc = 'size')

fuel_type,CNG,Diesel,Petrol
model,,,
M1,14948.0,NaN,NaN
M10,1209.0,NaN,NaN
M11,NaN,NaN,363.0
M2,NaN,NaN,1080.0
M3,NaN,NaN,2373.0
M4,NaN,14018.0,NaN
M5,NaN,1598.0,NaN
M6,NaN,NaN,13776.0
M7,NaN,NaN,2940.0


In [34]:
import lightgbm as lgbm

In [35]:
def catboost_model(train,test,Folds=20):
    x = train.drop(columns = ['policy_id','is_claim'])

    x['ratio1'] = x['age_of_car'] / x['policy_tenure']
    x['ratio2'] = x['age_of_policyholder'] / x['policy_tenure']

    test['ratio1'] = test['age_of_car'] / test['policy_tenure']
    test['ratio2'] = test['age_of_policyholder'] / test['policy_tenure']

    columns_to_drop = ['policy_tenure','population_density','make','segment','max_torque','max_power','engine_type', 'airbags', 'is_esc', 'is_adjustable_steering',
                   'is_tpms', 'is_parking_sensors', 'is_parking_camera','rear_brakes_type','displacement','cylinder','transmission_type','gear_box',
                   'steering_type','turning_radius','length','width', 'gross_weight', 'is_front_fog_lights', 'is_rear_window_wiper','is_rear_window_washer',
                   'is_rear_window_defogger', 'is_brake_assist', 'is_power_door_locks', 'is_central_locking', 'is_power_steering', 'is_driver_seat_height_adjustable',
                   'is_day_night_rear_view_mirror','is_ecw', 'is_speed_alert','ncap_rating']
    x.drop(columns = columns_to_drop,inplace=True)

    y = train['is_claim']
    x_test = test[x.columns.tolist()]

    print(x.dtypes)


    print(x.columns)

    cat_features = x.select_dtypes(include='object').columns.tolist()
    err = [] 
    oofs = np.zeros(shape=(len(x)))
    preds = np.zeros(shape=(len(test)))

    for col in cat_features:
      x[col] = pd.Categorical(x[col])
      x_test[col] = pd.Categorical(x_test[col])

    fold = StratifiedKFold(n_splits=Folds, shuffle=True, random_state=1)
    i = 1

    for train_index, test_index in fold.split(x, y):
        x_train, x_val = x.iloc[train_index], x.iloc[test_index]
        y_train, y_val = y.iloc[train_index], y.iloc[test_index]
        m = lgbm.LGBMClassifier( n_estimators=10000,learning_rate=0.1,num_leaves=3,categorical_feature = [2,3,4],
                     max_depth=10,reg_alpha = 10, reg_lambda = 10,random_state=42, early_stopping_round=50)  
        m.fit(x_train,y_train, eval_set = (x_val,y_val), verbose=200)
        print('HI')
        pred_y = m.predict_proba(x_val)[:,1]
        oofs[test_index] = pred_y

        precision, recall, thresholds = precision_recall_curve(y_val, pred_y)
        # convert to f score
        fscore = (2 * precision * recall) / (precision + recall)
        # locate the index of the largest f score
        ix = argmax(fscore)
        print(i, "Optimal F1: ", fscore[ix])
        print(i, "Optimal Threshold", thresholds[ix])
        err.append(fscore[ix])
        preds+= m.predict_proba(x_test)[:,1]
        # explainer = shap.TreeExplainer(m)
        # shap_values = explainer.shap_values(x_train)
        # shap.summary_plot(shap_values, x_train,plot_type='bar', max_display=20)
        i = i + 1
    preds=preds/Folds
    print(f"Average StratifiedKFold Score : {sum(err)/Folds} ")

    precision, recall, thresholds = precision_recall_curve(y, oofs)
    precision = np.where(precision ==0.0, 0.000000000000000001, precision)
    recall = np.where(recall ==0.0, 0.000000000000000001, recall)
        # convert to f score
    fscore = (2 * precision * recall) / (precision + recall)
        # locate the index of the largest f score
    ix = argmax(fscore)
    print("Final F Score", fscore[ix])
    print('Final Threshold', thresholds[ix])
    return preds
    # oof_score = rmsle(np.exp(oofs)-1,np.exp(y)-1)
    # print(f'\nOOF Auc is : {oof_score}')
    
    # oofs=pd.DataFrame(oofs,columns=['catboostoof'])
    # preds=pd.DataFrame(preds,columns=['catboostpred'])
    
    # oofs.to_csv('catboostoof.csv',index=False)
    # preds.to_csv('catboostpred.csv',index=False)


In [36]:
test['probs'] = catboost_model(train,test)

age_of_car             float64
age_of_policyholder    float64
area_cluster            object
model                   object
fuel_type               object
height                   int64
ratio1                 float64
ratio2                 float64
dtype: object
Index(['age_of_car', 'age_of_policyholder', 'area_cluster', 'model',
       'fuel_type', 'height', 'ratio1', 'ratio2'],
      dtype='object')



A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
Found `early_stopping_round` in params. Will use it instead of argument
categorical_feature keyword has been found in `params` and will be ignored.
Please use categorical_feature argument of the Dataset constructor to pass this parameter.
categorical_feature in param dict is overridden.


Training until validation scores don't improve for 50 rounds.
Early stopping, best iteration is:
[131]	valid_0's binary_logloss: 0.232497
HI
1 Optimal F1:  nan
1 Optimal Threshold 0.13198143471207602


invalid value encountered in true_divide
Found `early_stopping_round` in params. Will use it instead of argument
categorical_feature keyword has been found in `params` and will be ignored.
Please use categorical_feature argument of the Dataset constructor to pass this parameter.
categorical_feature in param dict is overridden.


Training until validation scores don't improve for 50 rounds.
[200]	valid_0's binary_logloss: 0.226172
[400]	valid_0's binary_logloss: 0.225593
Early stopping, best iteration is:
[500]	valid_0's binary_logloss: 0.225444
HI
2 Optimal F1:  nan
2 Optimal Threshold 0.18222880111944656


...

Training until validation scores don't improve for 50 rounds.
[200]	valid_0's binary_logloss: 0.229099
Early stopping, best iteration is:
[157]	valid_0's binary_logloss: 0.228902
HI
19 Optimal F1:  nan
19 Optimal Threshold 0.1360547920494441


invalid value encountered in true_divide
Found `early_stopping_round` in params. Will use it instead of argument
categorical_feature keyword has been found in `params` and will be ignored.
Please use categorical_feature argument of the Dataset constructor to pass this parameter.
categorical_feature in param dict is overridden.


Training until validation scores don't improve for 50 rounds.
[200]	valid_0's binary_logloss: 0.229129
[400]	valid_0's binary_logloss: 0.228274
Early stopping, best iteration is:
[459]	valid_0's binary_logloss: 0.228071
HI
20 Optimal F1:  nan
20 Optimal Threshold 0.17052292463760882


invalid value encountered in true_divide


Average StratifiedKFold Score : nan 
Final F Score 0.17392277560156685
Final Threshold 0.08716301213648549


In [37]:
test['is_claim'] = np.where(test['probs']>=0.08716301213648549, 1,0)

In [38]:
test['is_claim'].value_counts(normalize=True)

0    0.760003
1    0.239997
Name: is_claim, dtype: float64

In [39]:
test[['policy_id','is_claim']].to_csv('data_verse_13.csv',index=False)